In [14]:
import torch

m = torch.nn.LogSoftmax(dim=1)
input = torch.randn(3, 4)
output = m(input)

In [16]:
print(input)
print(output)

tensor([[-0.3946, -1.2349,  2.7990,  0.2295],
        [-2.8446, -0.1938,  0.0765, -1.1852],
        [ 0.1716, -2.1355,  1.0021, -0.2859]])
tensor([[-3.3205, -4.1608, -0.1269, -2.6963],
        [-3.6631, -1.0123, -0.7420, -2.0038],
        [-1.3929, -3.7001, -0.5625, -1.8505]])


In [37]:
import torch
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from data_utils.hulk import Hulk
import numpy as np

def remove_nan(im):
    non_nan_idx = ~np.isnan(im)
    min_image = np.min(im[non_nan_idx])
    im[~non_nan_idx] = min_image
    return im

transform = transforms.Compose([
    transforms.Lambda(remove_nan),
    transforms.ToTensor()
    ]
)
dataset = Hulk(targ_dir = "../4-HULK", transform = transform
               ,datalist="labeled.json")


dataloader = DataLoader(dataset=dataset, batch_size=64)

['COMPACT' 'DIFFUSE' 'DIFFUSE-LARGE' 'EXTENDED' 'FILAMENT' 'RADIO-GALAXY'
 'RING' 'ARTEFACT']
                                    source_name source_type survey telescope  \
count                                      4844        4844   4844      4844   
unique                                     4844          82      2         2   
top     G002.5+0.0IFx_Mosaic_Mom0_cutout0000001   [COMPACT]   mgps   meerkat   
freq                                          1        2734   2704      2704   

             project                                      original_path  \
count           4844                                               4844   
unique             3                                               4844   
top     lband-legacy  data/meerkat/G002/G002.5+0.0IFx_Mosaic_Mom0_cu...   
freq            2704                                                  1   

                                              target_path  \
count                                                4844   
unique 

/usr/local/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:878: UserWarning: unknown class(es) ['BACKGROUND', 'BORDER', 'MOSAICING', 'WTF'] will be ignored
  warnings.warn(


In [40]:
def get_mean_and_std(dataloader):
    channels_sum, channels_squared_sum, num_batches = 0, 0, 0
    for data, _ in dataloader:
        # Mean over batch, height and width, but not over the channels
        channels_sum += torch.mean(data, dim=[0,2,3])
        channels_squared_sum += torch.mean(data**2, dim=[0,2,3])
        num_batches += 1
    
    mean = channels_sum / num_batches

    # std = sqrt(E[X^2] - (E[X])^2)
    std = (channels_squared_sum / num_batches - mean ** 2) ** 0.5

    return mean, std

In [41]:
mean, std = get_mean_and_std(dataloader)

In [42]:
normalize = transforms.Normalize(mean=mean,
                                 std=std)



In [66]:

from matplotlib import pyplot as plt
import torchvision

#examples = next(iter(dataloader))

for batch  in dataloader:
    print(batch[0].shape)
    grid = torchvision.utils.make_grid(normalize(batch[0]))
    img = torchvision.transforms.ToPILImage()(grid)
    img.show()
    #examples = nex-t(iter(dataloader))
    #plt.imshow(grid)
    #plt.show()



torch.Size([64, 1, 256, 256])


TypeError: img should be Tensor Image. Got <class 'list'>